In [ ]:
# We construct an examiner toughness based on time
# First step: we need to be sure of the number of observations based on different dataset
# Application data: research-311404.Examiner_Toughness.Examiner_Application; USPTO-2021 PatEx DATASET, 13,556,609 entries
# Transaction data: research-311404.Examiner_Toughness.Examiner_transactions; USPTO-2021 PatEx DATASET, 481,133,923 entries
# Choose utility patents, and count the number of applications/amendments up to a date

In [1]:
import base64
import datetime
import pandas as pd
import json
from google.cloud import bigquery

client = bigquery.Client(location="US")
print("Client creating using default project: {}".format(client.project))

Client creating using default project: research-311404


In [3]:
# Consolidate to a single file
sql = """
        CREATE TABLE research-311404.Examiner_Toughness_Amendments_Consolidated.amendments
        AS SELECT t1.*, amendment_date FROM (SELECT application_number,filing_date,examiner_full_name,examiner_art_unit,patent_number FROM `research-311404.Examiner_Toughness.Examiner_Application` WHERE application_invention_type = 'Utility')t1
        LEFT JOIN
        (SELECT application_number,date AS amendment_date FROM `research-311404.Examiner_Toughness.Examiner_transactions` WHERE event_code = 'A...' or event_code = 'A.I.' or event_code = 'A.NE 'or event_code = 'A.NQ')t2
        ON t1.application_number = t2.application_number
        WHERE filing_date IS NOT NULL AND examiner_full_name IS NOT NULL
        ORDER BY examiner_full_name,filing_date,amendment_date
        """

job = client.query(sql)  # API request.
job.result()
# 13,281,623 outputs where there is not null for application year and examiner

In [4]:
# From this point on, we should construct two things: 1.cumulative count of applications for each examiner; 2. cumulative count of amendments for each examiner
# We start from the applications
# Consolidate to a single file
sql = """
        CREATE TABLE research-311404.Examiner_Toughness_Amendments_Consolidated.application_count
        AS SELECT 
            application_number, 
            filing_date, 
            examiner_full_name, 
            ROW_NUMBER() OVER(PARTITION BY examiner_full_name ORDER BY filing_date ASC) as application_count
        FROM 
             (SELECT application_number, filing_date, examiner_full_name FROM `research-311404.Examiner_Toughness_Amendments_Consolidated.amendments` GROUP BY application_number, filing_date, examiner_full_name)
        ORDER BY 
            examiner_full_name, 
            filing_date;

        """

job = client.query(sql)  # API request.
job.result()
# 9,992,041 unique applications

In [5]:
# Continue to amendments
# Consolidate to a single file
sql = """
        CREATE TABLE research-311404.Examiner_Toughness_Amendments_Consolidated.amendment_count
        AS SELECT 
    examiner_full_name, amendment_date,
        ROW_NUMBER() OVER(PARTITION BY examiner_full_name ORDER BY amendment_date ASC) as amendment_count
    FROM `research-311404.Examiner_Toughness_Amendments_Consolidated.amendments` WHERE amendment_date IS NOT NULL
    ORDER BY 
        examiner_full_name, 
        amendment_date;


        """

job = client.query(sql)  # API request.
job.result()
# 10,458,162 amendments

In [19]:
# Continue to amendments
# Consolidate to a single file
sql = """
SELECT examiner_full_name, filing_date  FROM research-311404.Examiner_Toughness_Amendments_Consolidated.application_count ORDER BY filing_date
"""
job = client.query(sql)  # API request.
df_A = job.to_dataframe()

#return the results as a pandas DataFrame.

sql = """
SELECT examiner_full_name,amendment_date,amendment_count FROM `research-311404.Examiner_Toughness_Amendments_Consolidated.amendment_count` ORDER BY amendment_date
"""
job = client.query(sql)  # API request.
df_B = job.to_dataframe()   

#return the results as a pandas DataFrame.

In [25]:
df_A['examiner_full_name'] = df_A['examiner_full_name'].astype("string")
df_B['examiner_full_name'] = df_B['examiner_full_name'].astype("string")

In [30]:
df_A['filing_date'] = pd.to_datetime(df_A['filing_date'], format='%Y-%m-%d')
df_B['amendment_date'] = pd.to_datetime(df_B['amendment_date'], format='%Y-%m-%d')

In [31]:
df_A = df_A.sort_values('filing_date')
df_B = df_B.sort_values('amendment_date')

In [32]:
df_B.dtypes

examiner_full_name    string[python]
amendment_date        datetime64[ns]
amendment_count                Int64
dtype: object

In [33]:
# Merge the dataframes
df_final = pd.merge_asof(df_A, df_B, left_on='filing_date', right_on='amendment_date', by='examiner_full_name', direction='backward')
# The final dataframe now contains the columns: examiner_full_name, filing_date, amendment_count

In [37]:
df = df_final.dropna()[['examiner_full_name','filing_date','amendment_count']]
# The project defaults to the Client's project if not specified.
dataset = client.get_dataset('research-311404.Examiner_Toughness_Amendments_Consolidated')  # API request
table_ref = dataset.table('amendment_count_by_filing_date')
job = client.load_table_from_dataframe(df, table_ref, location="US")
job.result()  # Waits for table load to complete.
print("Loaded dataframe to {}".format(table_ref.path))

Loaded dataframe to /projects/research-311404/datasets/Examiner_Toughness_Amendments_Consolidated/tables/amendment_count_by_filing_date


In [38]:
# With two things: 1.cumulative count of applications for each examiner; 2. cumulative count of amendments for each examiner
# We calculate for each examiner and filing-date 
# Consolidate to a single file
# eliminate all patents assigned to ' Central Docket'
sql = """
        CREATE TABLE research-311404.Examiner_Toughness_Amendments_Consolidated.full_count
        AS SELECT application_number,t1.filing_date,t1.examiner_full_name,application_count,amendment_count
         FROM (SELECT 
                    * FROM research-311404.Examiner_Toughness_Amendments_Consolidated.application_count)t1
                    LEFT JOIN
                     (SELECT examiner_full_name, filing_date, amendment_count FROM `research-311404.Examiner_Toughness_Amendments_Consolidated.amendment_count_by_filing_date`)t2
                     ON t1.examiner_full_name = t2.examiner_full_name AND t1.filing_date = t2.filing_date
                     WHERE t1.examiner_full_name != 'CENTRAL, DOCKET'
                     GROUP BY application_number,t1.filing_date,t1.examiner_full_name,application_count,amendment_count


        """

job = client.query(sql)  # API request.
job.result()
# 9,695,836 unique applications, central docket applications are no longer considered

In [39]:
# We need the plain toughness and scale it by art unit
# scale it by the overall examiner toughness OF THAT YEAR!

# art_unit_year level toughness    
sql = """
    CREATE TABLE research-311404.Examiner_Toughness_Amendments_Consolidated.art_unit_toughness
    AS SELECT filing_year,examiner_art_unit,IFNULL(SUM(amendment_count)/SUM(application_count),0) as toughness FROM (SELECT application_number,LEFT(CAST(filing_date AS STRING),4) AS filing_year,examiner_full_name,application_count,amendment_count
    FROM research-311404.Examiner_Toughness_Amendments_Consolidated.full_count)t1
    LEFT JOIN
    (SELECT application_number,examiner_art_unit FROM `research-311404.Examiner_Toughness.Examiner_Application`)t2
    ON t1.application_number = t2.application_number
    WHERE examiner_art_unit IS NOT NULL
    GROUP BY filing_year,examiner_art_unit
    """

job = client.query(sql)  # API request.
job.result()
# 23,044 art unit - year level toughness

In [40]:
# patent level toughness    
sql = """
    CREATE TABLE research-311404.Examiner_Toughness_Amendments_Consolidated.examiner_toughness
    AS SELECT patent_number,filing_year,examiner_art_unit,IFNULL(amendment_count/application_count,0) AS examiner_toughness FROM (SELECT application_number,LEFT(CAST(filing_date AS STRING),4) AS filing_year,examiner_full_name,application_count,amendment_count
    FROM research-311404.Examiner_Toughness_Amendments_Consolidated.full_count)t1
    LEFT JOIN
    (SELECT application_number,examiner_art_unit,patent_number FROM `research-311404.Examiner_Toughness.Examiner_Application`)t2
    ON t1.application_number = t2.application_number
    WHERE examiner_art_unit IS NOT NULL AND patent_number IS NOT NULL
    """

job = client.query(sql)  # API request.
job.result()
# 7,051,680 patents. Because limitations from the OCE PAIR dataset



In [41]:
# scaled examiner level toughness    
sql = """
    CREATE TABLE research-311404.Examiner_Toughness_Amendments_Consolidated.examiner_scaled_toughness
    AS SELECT patent_number,t1.filing_year,IFNULL(examiner_toughness/NULLIF(toughness,0),0) AS scaled_toughness FROM (SELECT *
    FROM  research-311404.Examiner_Toughness_Amendments_Consolidated.examiner_toughness)t1
    LEFT JOIN
    (SELECT filing_year,examiner_art_unit,toughness
    FROM `research-311404.Examiner_Toughness_Amendments_Consolidated.art_unit_toughness`)t2
    ON t1.filing_year= t2.filing_year AND t1.examiner_art_unit = t2.examiner_art_unit

    """

job = client.query(sql)  # API request.
job.result()
# 7,051,680 patents. Because limitations from the OCE PAIR dataset